In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!sudo apt install build-essential

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [3]:
%pip install -r "/content/drive/MyDrive/Protein-binding/requirements.txt"
%pip install datasets mdtraj dssp
%pip install numpy --no-cache-dir
%pip install pandas==2.2.0

ERROR: Could not find a version that satisfies the requirement dssp (from versions: none)
ERROR: No matching distribution found for dssp


In [4]:
import torch
import ast
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.nn.functional as F
import warnings
import mdtraj as md
import esm
import gc
import pickle
import torch.optim as optim

from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, global_add_pool
from Bio import SeqIO
from Bio import PDB
from Bio.PDB.DSSP import DSSP
from Bio.PDB.Polypeptide import is_aa
from Bio.SeqUtils import seq1
from torch_geometric.data import Data
from transformers import (AutoModelForTokenClassification, AutoTokenizer,
                          AutoModelForMaskedLM, DataCollatorForTokenClassification,
                           EsmForMaskedLM, EsmTokenizer, EsmModel, EsmForTokenClassification,
                           TrainingArguments, Trainer, TrainerCallback
                        )
from transformers.trainer_callback import ProgressCallback
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support,
                             matthews_corrcoef, roc_auc_score)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from pprint import pprint
from datasets import Dataset
from datetime import datetime
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from glob import glob
from loguru import logger

warnings.filterwarnings("ignore", message="Ignoring unrecognized record 'END'")

### Preparing train-test dataset

In [5]:
initial_train_df = pd.read_csv("/content/drive/MyDrive/Protein-binding/data/development_set/full_grouped_train_binding_sites_df.csv")
initial_train_df['binding_sites'] = initial_train_df['binding_sites'].apply(ast.literal_eval)
initial_train_df['any_ligand_binding_sites'] = initial_train_df['any_ligand_binding_sites'].apply(ast.literal_eval)
initial_train_df['metal_binding'] = initial_train_df['metal_binding'].apply(ast.literal_eval)
initial_train_df['small_binding'] = initial_train_df['small_binding'].apply(ast.literal_eval)
initial_train_df['nuclear_binding'] = initial_train_df['nuclear_binding'].apply(ast.literal_eval)

In [6]:
initial_train_df

,prot_id,binding_sites,ligand_type,sequence,sequence_length,metal_binding,small_binding,nuclear_binding,any_ligand_binding_sites
0,A0A0F7RDM3,"[29, 53, 129, 135, 137]",['metal'],MEIRKKLVVPSKYGTKCPYTMKPKYITVHNTYNDAPAENEVNYMIT...,234,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,A0A0F7RHX8,"[67, 72, 76, 78, 123, 133, 135]",['small'],MHLKEKITTIIQGQRTGVLSTVRNDKPHSAFMMFFHEDFVLYVATD...,138,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,A0A0H2W6Y8,"[85, 119, 125, 126, 129, 144, 197, 198, 207, 2...",['metal'],MMKILGLIGGMSWESTIPYYRMINQHVKAQLGGLHSAKIILYSVDF...,231,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,A0A2B6C3P9,"[79, 111, 114, 117, 122, 125]",['metal'],MSINKWLFRFIGFLVMLVVITTLNSLNVFASVNDLAQPIASAKVIE...,129,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,A0A384LKY8,"[5, 10, 12, 14, 43, 63, 72, 74, 76, 83, 85, 11...","['metal', 'small']",MSEHFVGKYEVELKFRVMDLTTLHEQLVAQKATAFTLNNHEKDIYL...,179,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, ..."
...,...,...,...,...,...,...,...,...,...
1009,S0BAP9,"[56, 66, 69, 70, 72, 73, 77, 80, 81, 86, 89, 9...",['small'],MNPLSTVLLVLCATSAALASEFCSEADATIVIKQWNQIYNAGIGAK...,168,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1010,S3TFW2,"[75, 79, 122, 192, 193, 204, 277, 278, 279, 28...",['small'],MTRPDSKSMNYQLLKTFSRQPIQFGRFLARLLAGLVNTLKITRTSK...,327,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1011,V6F235,"[230, 232, 255]",['small'],MRKSGCAVCSRSIGWVGLAVSTVLMVMKAFVGLIGGSQAMLADAMY...,318,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1012,V9P0A9,"[13, 33, 34, 37, 38, 42, 49, 51, 70, 72, 74, 7...",['small'],MADFKFEPMRSLIYVDCVSEDYRPKLQRWIYKVHIPDSISQFEPYV...,283,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [7]:
prot_id = 'Q9NZV6'
excluded_prot_id_index = index = initial_train_df.index[initial_train_df['prot_id'] == prot_id].tolist()[0]
print(excluded_prot_id_index)

962


In [8]:
np.random.seed(42)
excluded_protein_id = ['Q9NZV6']

train_df = initial_train_df[~initial_train_df['prot_id'].isin(excluded_protein_id)]

# train_df, val_df = train_test_split(
#     needed_training_sites_df,
#     test_size = 0.1,  # 90% train, 10% validation
#     stratify = None,
#     random_state = 42
# )

In [9]:
test_df = pd.read_csv("/content/drive/MyDrive/Protein-binding/data/development_set/full_grouped_test_binding_sites_df.csv")
# test_df = pd.read_csv("/content/drive/MyDrive/Protein-binding/data/independent_set/grouped_test_46_new_binding_sites.csv")

test_df['binding_sites'] = test_df['binding_sites'].apply(ast.literal_eval)
test_df['any_ligand_binding_sites'] = test_df['any_ligand_binding_sites'].apply(ast.literal_eval)
test_df['metal_binding'] = test_df['metal_binding'].apply(ast.literal_eval)
test_df['small_binding'] = test_df['small_binding'].apply(ast.literal_eval)
test_df['nuclear_binding'] = test_df['nuclear_binding'].apply(ast.literal_eval)

In [10]:
def split_into_chunks(sequences, labels, chunk_size = 1000):
    """Split sequences and labels into chunks of size "chunk_size" or less."""
    new_sequences = []
    new_labels = []
    for seq, lbl in zip(sequences, labels):
        if len(seq) > chunk_size:
            # Split the sequence and labels into chunks of size "chunk_size" or less
            for i in range(0, len(seq), chunk_size):
                new_sequences.append(seq[i:i+chunk_size])
                new_labels.append(lbl[i:i+chunk_size])
        else:
            new_sequences.append(seq)
            new_labels.append(lbl)

    return new_sequences, new_labels

In [11]:
# Initial sequences
test_seq = test_df['sequence'].tolist()
test_labels = test_df['metal_binding'].tolist() # Edit labels

train_seq = train_df['sequence'].tolist()
train_labels = train_df['metal_binding'].tolist() # Edit labels

# val_seq = val_df['sequence'].tolist()
# val_labels = val_df['any_ligand_binding_sites'].tolist()

# Apply new sequences by chunking
chunk_size = 1000

test_seq, test_labels = split_into_chunks(test_seq, test_labels, chunk_size)
train_seq, train_labels = split_into_chunks(train_seq, train_labels, chunk_size)
# val_seq, val_labels = split_into_chunks(val_seq, val_labels, chunk_size)

### Get embeddings from ESM-3 language model

In [12]:
# train_sequences_fold_2_esm_embeddings = np.load('/content/drive/MyDrive/Protein-binding/train_sequences_esm3_medium_embeddings_Apr_13_601_to_1013.npy', allow_pickle=True)
# train_sequences_fold_1_esm_embeddings = np.load('/content/drive/MyDrive/Protein-binding/train_sequences_esm3_medium_embeddings_Apr_13_0_to_600.npy', allow_pickle=True)

In [13]:
# train_sequences_fold_2_esm_embeddings = np.delete(train_sequences_fold_2_esm_embeddings, 361, axis=0)  # Note: Python uses 0-based indexing, so 962nd element is at index 961.

In [14]:
# esm3_batch_size = 50

# combined_embeddings = np.concatenate([train_sequences_fold_1_esm_embeddings, train_sequences_fold_2_esm_embeddings])
# num_batches = len(combined_embeddings) // esm3_batch_size + (len(combined_embeddings) % esm3_batch_size > 0)  # Calculate the number of batches

# for i in range(num_batches):
#     start = i * esm3_batch_size
#     end = min((i + 1) * esm3_batch_size, len(combined_embeddings))  # Ensure you don't go beyond the array bounds
#     batch = combined_embeddings[start:end]
#     np.save(f'/content/drive/MyDrive/Protein-binding/esm-3-embeddings/combined_embeddings_batch_{i+1}.npy', batch)

In [15]:
# test_sequences_esm_embeddings = np.load('/content/drive/MyDrive/Protein-binding/test_sequences_esm3_medium_embeddings_Apr_14.npy', allow_pickle=True)

In [16]:
# test_num_batches = len(test_sequences_esm_embeddings) // esm3_batch_size + (len(test_sequences_esm_embeddings) % esm3_batch_size > 0)  # Calculate the number of batches

# for i in range(test_num_batches):
#     start = i * esm3_batch_size
#     end = min((i + 1) * esm3_batch_size, len(test_sequences_esm_embeddings))  # Ensure you don't go beyond the array bounds
#     batch = test_sequences_esm_embeddings[start:end]
#     np.save(f'/content/drive/MyDrive/Protein-binding/esm-3-embeddings/test300_embeddings_batch_{i+1}.npy', batch)

In [17]:

# tokenizer = EsmTokenizer.from_pretrained(pretrained_model)
# max_sequence_length = 1000

# train_tokenized = tokenizer(train_seq, padding=True, truncation=True, max_length=max_sequence_length, return_tensors="pt", is_split_into_words=False)
# test_tokenized = tokenizer(test_seq, padding=True, truncation=True, max_length=max_sequence_length, return_tensors="pt", is_split_into_words=False)
# # eval_tokenized = tokenizer(val_seq, padding=True, truncation=True, max_length=max_sequence_length, return_tensors="pt", is_split_into_words=False)

In [18]:
train_num_batches = 21
test_num_batches = 6

In [19]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
gc.collect()  # Force garbage collection to potentially free up memory
torch.cuda.empty_cache()  # Empty the CUDA cache

In [20]:
train_embeddings = []
test_embeddings = []

for i in tqdm(range(1, train_num_batches + 1)):
    train_batch = np.load(f'/content/drive/MyDrive/Protein-binding/esm-3-embeddings/combined_embeddings_batch_{i}.npy', allow_pickle=True)
    train_batch_embeddings = [ele.embeddings for ele in train_batch]
    train_batch_embeddings = [np.squeeze(emb) for emb in train_batch_embeddings]
    train_embeddings.extend(train_batch_embeddings)

for i in tqdm(range(1, test_num_batches + 1)):
    test_batch = np.load(f'/content/drive/MyDrive/Protein-binding/esm-3-embeddings/test300_embeddings_batch_{i}.npy', allow_pickle=True)
    test_batch_embeddings = [ele.embeddings for ele in test_batch]
    test_batch_embeddings = [np.squeeze(emb) for emb in test_batch_embeddings]
    test_embeddings.extend(test_batch_embeddings)

100%|██████████| 6/6 [01:12<00:00, 12.11s/it]


In [21]:
sample_batch = np.load(f'/content/drive/MyDrive/Protein-binding/esm-3-embeddings/combined_embeddings_batch_{i}.npy', allow_pickle=True)
sample_train_batch_embeddings = [ele.embeddings for ele in sample_batch]
sample_train_batch_embeddings = [np.squeeze(emb) for emb in sample_train_batch_embeddings]

print(sample_train_batch_embeddings[0].shape)

torch.Size([73, 1152])


### Features extraction

In [22]:
def get_structure(prot_id, pdb_file):
    parser = PDB.PDBParser()
    structure = parser.get_structure(prot_id, pdb_file)
    return structure

def extract_coordinates(structure):
    # Extract Cα coordinates (central carbon atom)
    coordinates = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if "CA" in residue:  # Get Cα atom
                    ca_atom = residue["CA"]
                    coord = ca_atom.get_coord()  # Returns numpy array [x, y, z]
                    coordinates.append(coord)

    return coordinates

def get_amino_acid_types(structure):
    amino_acids = []
    for model in structure:
        for chain in model:
            for residue in chain:
                # Get residue name (3-letter code)
                if not is_aa(residue):
                    continue
                res_name = residue.get_resname()
                # Convert to 1-letter code if needed
                one_letter = seq1(res_name)
                amino_acids.append(one_letter)
    return amino_acids

def get_secondary_structure_mdtraj(pdb_file, sequence_length):
    """
    Extract secondary structure features from a .PDB file using mdtraj.

    Args:
        pdb_file (str): Path to the .PDB file.
        sequence_length (int): Length of the protein sequence.

    Returns:
        dict: A dictionary with:
            - 'raw': List of raw secondary structure codes.
            - 'one_hot': Tensor of one-hot encoded secondary structure.
    """
    # Load the .PDB file with mdtraj
    traj = md.load(pdb_file)

    # Compute secondary structure
    ss = md.compute_dssp(traj)[0]  # Returns codes like 'H', 'E', 'C', '-'('NA')

    # One-hot encode
    ss_onehot = []
    for code in ss:
        ss_onehot.append([
            1 if code == 'H' else 0,
            1 if code == 'E' else 0,
            1 if code == 'T' else 0,
            1 if code == 'C' or code == 'NA' else 0
        ])

    ss_onehot = torch.tensor(ss_onehot, dtype=torch.float32)

    # Pad with zeros to match sequence length
    if ss_onehot.shape[0] < sequence_length:
        padding = torch.zeros((sequence_length - ss_onehot.shape[0], ss_onehot.shape[1]), dtype=torch.float32)
        ss_onehot = torch.cat([ss_onehot, padding], dim=0)

    return {
        "raw": ss.tolist(),
        "one_hot": ss_onehot
    }

def calculate_residue_distances(coordinates):
    """
    Calculate pairwise distances between residues in a protein structure.
    Args: Coordinates (list): List of residue atom's coordinates.
    Returns:
        np.ndarray: 2D array of pairwise distances.
    """
    num_residues = len(coordinates)
    distances = np.zeros((num_residues, num_residues))

    for i in range(num_residues):
        for j in range(i + 1, num_residues):
            dist = np.linalg.norm(coordinates[i] - coordinates[j])
            distances[i, j] = distances[j, i] = dist

    return distances

In [23]:
def get_dihedral_angles(pdb_file, sequence_length):
    traj = md.load(pdb_file)
    # Compute phi and psi angles
    phi_indices, phi_angles = md.compute_phi(traj)
    psi_indices, psi_angles = md.compute_psi(traj)

    # Convert to degrees and create tensors
    phi_angles = torch.tensor(np.degrees(phi_angles[0]), dtype=torch.float32).unsqueeze(1)  # Shape: [num_residues-1, 1]
    psi_angles = torch.tensor(np.degrees(psi_angles[0]), dtype=torch.float32).unsqueeze(1)  # Shape: [num_residues-1, 1]

    # Pad with zeros to match sequence length
    while phi_angles.shape[0] < sequence_length:
        phi_angles = torch.cat([phi_angles, torch.zeros(1, 1, dtype=torch.float32)], dim=0)
    while psi_angles.shape[0] < sequence_length:
        psi_angles = torch.cat([psi_angles, torch.zeros(1, 1, dtype=torch.float32)], dim=0)

    return phi_angles, psi_angles


def get_b_factors(structure, sequence_length):
    b_factors = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if "CA" in residue:
                    ca_atom = residue["CA"]
                    b_factor = ca_atom.get_bfactor()
                    b_factors.append(b_factor)

    # Pad with zeros if b_factors length is less than sequence length
    while len(b_factors) < sequence_length:
        b_factors.append(0.0)

    return torch.tensor(b_factors, dtype=torch.float32).unsqueeze(1)

In [24]:
# Compute RSA values for amino acids
MAX_SASA = {
    'ALA': 129.0, 'ARG': 274.0, 'ASN': 195.0, 'ASP': 193.0, 'CYS': 167.0,
    'GLU': 223.0, 'GLN': 225.0, 'GLY': 104.0, 'HIS': 224.0, 'ILE': 197.0,
    'LEU': 201.0, 'LYS': 236.0, 'MET': 224.0, 'PHE': 240.0, 'PRO': 159.0,
    'SER': 155.0, 'THR': 172.0, 'TRP': 285.0, 'TYR': 263.0, 'VAL': 174.0
}

def compute_rsa(structure_file, chain_id=None):
    """
    Compute RSA for each residue in a protein structure.

    Args:
        structure_file (str): Path to the protein structure file (e.g., PDB file).
        chain_id (str, optional): Chain ID to analyze (if None, uses the first chain).

    Returns:
        list: List of (residue_name, residue_id, rsa) tuples.
    """
    traj = md.load(structure_file)

    if chain_id:
        chain = next(c for c in traj.topology.chains if c.chain_id == chain_id)
        traj = traj.atom_slice([atom.index for atom in traj.topology.atoms if atom.residue.chain == chain])

    sasa = md.shrake_rupley(traj, mode='residue')[0]  # Shape: [n_residues]
    rsa_values = []
    for i, residue in enumerate(traj.topology.residues):
        if not residue.is_protein:
            continue
        res_name = residue.name
        res_id = residue.resSeq
        residue_sasa = sasa[i]
        max_sasa = MAX_SASA.get(res_name, 0.0)
        if max_sasa == 0.0:
            print(f"Warning: No max SASA value for residue {res_name}. Skipping.")
            continue
        rsa = residue_sasa / max_sasa if max_sasa > 0 else 0.0
        rsa = min(rsa, 1.0)
        rsa_values.append((res_name, res_id, rsa))
    return rsa_values



In [25]:
def fuse_features(esm3_embeddings, ss_onehot, phi_angles, psi_angles, b_factors):
    """
    Fuse ESM-2 embeddings with structural features.

    Args:
        esm3_embeddings (torch.Tensor): Shape [num_residues, 1280]
        ss_onehot (torch.Tensor): Shape [num_residues, 4]
        phi_angles (torch.Tensor): Shape [num_residues, 1]
        psi_angles (torch.Tensor): Shape [num_residues, 1]
        b_factors (torch.Tensor): Shape [num_residues, 1]

    Returns:
        torch.Tensor: Fused node features, shape [num_residues, 1159] (1152 + 4 + 1 + 1 + 1)
    """
    # Ensure all features have the same length
    num_residues = esm3_embeddings.shape[0]

    # Print shapes for debugging
    print(f"esm3_embeddings shape: {esm3_embeddings.shape}")
    print(f"ss_onehot shape: {ss_onehot.shape}")
    print(f"phi_angles shape: {phi_angles.shape}")
    print(f"psi_angles shape: {psi_angles.shape}")
    print(f"b_factors shape: {b_factors.shape}")

    # Adjust ss_onehot, phi_angles, psi_angles, b_factors if needed
    min_length = min(num_residues, ss_onehot.shape[0], phi_angles.shape[0], psi_angles.shape[0], b_factors.shape[0])

    esm3_embeddings = esm3_embeddings[:min_length]
    ss_onehot = ss_onehot[:min_length]
    phi_angles = phi_angles[:min_length]
    psi_angles = psi_angles[:min_length]
    b_factors = b_factors[:min_length]

    # Concatenate all features
    node_features = torch.cat([
        esm3_embeddings,  # [num_residues, 1280]
        ss_onehot,        # [num_residues, 4]
        phi_angles,       # [num_residues, 1]
        psi_angles,       # [num_residues, 1]
        b_factors,        # [num_residues, 1]
    ], dim=1)  # Shape: [num_residues, 1287]

    return node_features

In [26]:
# def create_edge_features(distances, threshold=8.0):
#     num_residues = distances.shape[0]
#     contact_map = (distances < threshold) & (distances > 0)
#     edge_index = torch.nonzero(torch.tensor(contact_map, dtype=torch.bool), as_tuple=False).t()  # Shape: [2, num_edges]

#     # Vectorized edge attribute computation
#     src, dst = edge_index[0], edge_index[1]
#     dists = torch.tensor(distances[src, dst], dtype=torch.float32)
#     seq_seps = torch.abs(src - dst).float()

#     # Normalize features
#     dists = dists / threshold  # Scale to 0–1
#     seq_seps = seq_seps / num_residues  # Scale to 0–1

#     edge_attr = torch.stack([dists, seq_seps], dim=1)  # Shape: [num_edges, 2]

#     return edge_index, edge_attr

def create_edge_features(distances, threshold=8.0):
    num_residues = distances.shape[0]
    contact_map = (distances < threshold) & (distances > 0)

    # Restrict indices to be within the valid node range (0 to num_residues - 1)
    edge_index = torch.nonzero(torch.tensor(contact_map, dtype=torch.bool), as_tuple=False).t()
    edge_index = edge_index[:, :num_residues]  # Ensure indices are within bounds

    # Vectorized edge attribute computation
    src, dst = edge_index[0], edge_index[1]
    dists = torch.tensor(distances[src, dst], dtype=torch.float32)
    seq_seps = torch.abs(src - dst).float()

    # Normalize features
    dists = dists / threshold  # Scale to 0–1
    seq_seps = seq_seps / num_residues  # Scale to 0–1

    edge_attr = torch.stack([dists, seq_seps], dim=1)  # Shape: [num_edges, 2]

    return edge_index, edge_attr


In [27]:
def create_graph_data(node_features, edge_index, edge_attr, labels):
    """
    Create a PyTorch Geometric Data object for the protein graph.

    Args:
        node_features (torch.Tensor): Shape [num_residues, 1288], fused node features.
        edge_index (torch.Tensor): Shape [2, num_edges], indices of connected nodes.
        edge_attr (torch.Tensor): Shape [num_edges, 2], edge features.
        labels (torch.Tensor): Shape [num_residues], binary labels (0 or 1) for binding sites.

    Returns:
        Data: PyTorch Geometric Data object.
    """
    return Data(
        x=node_features,      # Node features
        edge_index=edge_index,  # Edge indices
        edge_attr=edge_attr,  # Edge features
        y=labels              # Labels for binding site prediction
    )

### Test feature extraction

In [28]:
sample_idx = 5
sample_prot_id = train_df.iloc[sample_idx]['prot_id']
sample_sequence = train_df.iloc[sample_idx]['sequence']
sample_sequence_len = len(train_df.iloc[sample_idx]['sequence'])
sample_labels = train_df.iloc[sample_idx]['any_ligand_binding_sites']
sample_train_embeddings = train_embeddings[sample_idx]

sample_structure_file = f"/content/drive/MyDrive/Protein-binding/esmFold_pdb_files/{sample_prot_id}.pdb"

sample_structure = get_structure(sample_prot_id, sample_structure_file)
sample_coordinates = extract_coordinates(sample_structure)
sample_distances = calculate_residue_distances(sample_coordinates)
sample_ss_one_hot = get_secondary_structure_mdtraj(sample_structure_file, sample_sequence_len)['one_hot'].to(device)
sample_phi_angles, sample_psi_angles = get_dihedral_angles(sample_structure_file, sample_sequence_len)
sample_b_factors = get_b_factors(sample_structure, sample_sequence_len).to(device)
sample_rsa_values = compute_rsa(sample_structure_file)
min_length = 1

print(sample_rsa_values)
rsa_dict = {res_id: rsa for _, res_id, rsa in sample_rsa_values}
sample_rsa_tensor = torch.zeros((min_length, 1), dtype=torch.float)
print(sample_rsa_tensor)

# for i, res_id in enumerate(residue_ids[:min_length]):
#     sample_rsa_tensor[i, 0] = rsa_dict.get(res_id, 0.0)

# print(sample_prot_id)
# print(sample_sequence)
# print(len(sample_sequence))
# print(sample_b_factors.shape)
# print(sample_ss_one_hot.shape)
# print(sample_phi_angles.shape)
# print(sample_psi_angles.shape)
# print(sample_rsa_tensor.shape)

[('MET', 1, 0.010369594608034407), ('THR', 2, 0.007762903390928756), ('GLU', 3, 0.007529691730379524), ('THR', 4, 0.006649154563282811), ('THR', 5, 0.00706053750459538), ('THR', 6, 0.00409893067770226), ('THR', 7, 0.0042963630931321965), ('PHE', 8, 0.00474682201941808), ('MET', 9, 0.004713817898716245), ('ASP', 10, 0.0037179122934687323), ('ASN', 11, 0.004707534496600811), ('VAL', 12, 0.0014951566512557283), ('LEU', 13, 0.0038864903189056547), ('GLY', 14, 0.0029046317705741296), ('TRP', 15, 0.005465454804269891), ('LEU', 16, 0.0009672351885790848), ('HIS', 17, 0.005670112158570971), ('LYS', 18, 0.007587622787992833), ('GLY', 19, 0.005250471142622141), ('TYR', 20, 0.00211792119102333), ('PRO', 21, 0.006458016311597524), ('GLU', 22, 0.007889507597337388), ('GLY', 23, 0.004039539167514214), ('VAL', 24, 0.0024281579187546655), ('PRO', 25, 0.002634566347554045), ('PRO', 26, 0.006291117308274755), ('LYS', 27, 0.0070374547424962965), ('ASP', 28, 0.002006227179512459), ('TYR', 29, 0.0044379356

In [29]:
print(sample_train_embeddings.shape)

torch.Size([105, 1152])


In [30]:
edge_index, edge_attr = create_edge_features(sample_distances)
node_features = fuse_features(sample_train_embeddings.to(device), sample_ss_one_hot,
                              sample_phi_angles.to(device), sample_psi_angles.to(device), sample_b_factors)
labels = torch.tensor(sample_labels, dtype = torch.long).to(device)

esm3_embeddings shape: torch.Size([105, 1152])
ss_onehot shape: torch.Size([103, 4])
phi_angles shape: torch.Size([103, 1])
psi_angles shape: torch.Size([103, 1])
b_factors shape: torch.Size([103, 1])


In [31]:
graph_data = create_graph_data(node_features, edge_index, edge_attr, labels)

In [32]:
# print(len(train_df))
# print(len(train_labels))
# print(len(train_seq))

### Prepare data tensors

In [33]:
def get_graph_data(df, embeddings, device):
    graph_data_list = []

    for idx in tqdm(range(len(df))):
        prot_id = df.iloc[idx]['prot_id']
        sequence = df.iloc[idx]['sequence']
        sequence_len = len(df.iloc[idx]['sequence'])
        labels = df.iloc[idx]['any_ligand_binding_sites']
        embedding = embeddings[idx]

        try:
            structure_file = f"/content/drive/MyDrive/Protein-binding/esmFold_pdb_files/{prot_id}.pdb"
            structure = get_structure(prot_id, structure_file)
            coordinates = extract_coordinates(structure)
            distances = calculate_residue_distances(coordinates)
            # print(f"Distances: {distances}")
            ss_one_hot = get_secondary_structure_mdtraj(structure_file, sequence_len)['one_hot'].to(device)
            phi_angles, psi_angles = get_dihedral_angles(structure_file, sequence_len)
            b_factors = get_b_factors(structure, sequence_len).to(device)
        except Exception as e:
            # print(f"PDB file of {prot_id} ID cannot be found")
            print(f"Error: {e}")

        edge_index, edge_attr = create_edge_features(distances)
        node_features = fuse_features(embedding.to(device), ss_one_hot,
                                    phi_angles.to(device), psi_angles.to(device), b_factors)
        labels = torch.tensor(labels, dtype = torch.long).to(device)
        print(f"Shape of labels: {labels.shape}")
        graph_data = create_graph_data(node_features, edge_index, edge_attr, labels)

        graph_data_list.append(graph_data)

    return graph_data_list

In [34]:
train_graphs_data = get_graph_data(train_df, train_embeddings, device)
test_graphs_data = get_graph_data(test_df, test_embeddings, device)

  0%|          | 1/1013 [00:00<06:24,  2.63it/s]

esm3_embeddings shape: torch.Size([236, 1152])
ss_onehot shape: torch.Size([234, 4])
phi_angles shape: torch.Size([234, 1])
psi_angles shape: torch.Size([234, 1])
b_factors shape: torch.Size([234, 1])
Shape of labels: torch.Size([234])


  0%|          | 2/1013 [00:00<05:20,  3.15it/s]

esm3_embeddings shape: torch.Size([140, 1152])
ss_onehot shape: torch.Size([138, 4])
phi_angles shape: torch.Size([138, 1])
psi_angles shape: torch.Size([138, 1])
b_factors shape: torch.Size([138, 1])
Shape of labels: torch.Size([138])


  0%|          | 3/1013 [00:01<06:33,  2.57it/s]

...

esm3_embeddings shape: torch.Size([133, 1152])
ss_onehot shape: torch.Size([131, 4])
phi_angles shape: torch.Size([131, 1])
psi_angles shape: torch.Size([131, 1])
b_factors shape: torch.Size([131, 1])
Shape of labels: torch.Size([131])


100%|█████████▉| 299/300 [02:33<00:00,  1.94it/s]

esm3_embeddings shape: torch.Size([243, 1152])
ss_onehot shape: torch.Size([241, 4])
phi_angles shape: torch.Size([241, 1])
psi_angles shape: torch.Size([241, 1])
b_factors shape: torch.Size([241, 1])
Shape of labels: torch.Size([241])


100%|██████████| 300/300 [02:33<00:00,  1.95it/s]

esm3_embeddings shape: torch.Size([154, 1152])
ss_onehot shape: torch.Size([152, 4])
phi_angles shape: torch.Size([152, 1])
psi_angles shape: torch.Size([152, 1])
b_factors shape: torch.Size([152, 1])
Shape of labels: torch.Size([152])


### Create baseline GNN

In [35]:
class BindingSiteGCN(nn.Module):
    def __init__(self, node_dim=1159, edge_dim=2, hidden_dim=512):
        """
        A baseline GCN for binding site prediction.

        Args:
            node_dim (int): Dimension of node features (1288 in your case).
            edge_dim (int): Dimension of edge features (2 in your case).
            hidden_dim (int): Hidden dimension for GCN layers.
        """
        super(BindingSiteGCN, self).__init__()
        # GCN layers with edge features
        self.conv1 = GCNConv(node_dim, hidden_dim)  # Remove edge_dim
        self.conv2 = GCNConv(hidden_dim, 256)  # Remove edge_dim
        self.conv3 = GCNConv(256, 128)  # Remove edge_dim

        # Linear layer to process edge features
        self.edge_lin = nn.Linear(edge_dim, hidden_dim)

        # Final classifier
        self.fc = nn.Linear(128, 2)  # 2 classes: 0 (non-binding), 1 (binding)

        # Activation and dropout
        self.activation_func = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.1)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        # Get the number of nodes in the current batch

        # Process edge features separately
        edge_attr = self.edge_lin(edge_attr)

        # GCN layers
        # Pass num_nodes to gcn_norm
        x = self.conv1(x, edge_index)  # Removed edge_attr here, Added num_nodes
        x = self.activation_func(x)
        x = self.dropout(x)

        x = self.conv2(x, edge_index) # Removed edge_attr here, Added num_nodes
        x = self.activation_func(x)
        x = self.dropout(x)

        x = self.conv3(x, edge_index) # Removed edge_attr here, Added num_nodes
        x = self.activation_func(x)

        # Final classification
        x = self.fc(x)  # Shape: [num_residues, 2]

        return x  # Logits for each residue

In [36]:
train_graphs, val_graphs = train_test_split(
    train_graphs_data,
    test_size=0.05,
    random_state=42,
    shuffle=True
)

print(f"Number of training graphs: {len(train_graphs)}")
print(f"Number of validation graphs: {len(val_graphs)}")
print(f"Number of testing graphs: {len(test_graphs_data)}")

Number of training graphs: 962
Number of validation graphs: 51
Number of testing graphs: 300


In [37]:
for idx, train_graph in enumerate(train_graphs):
    if train_graph.x.shape[0] != train_graph.y.shape[0]:
        print(f"Abnormal at index: {idx}")
        print(f"Train graph's input shape: {train_graph['x'].shape[0]} and labels shape: {train_graph['y'].shape[0]}")

Abnormal at index: 0
Train graph's input shape: 171 and labels shape: 169


In [38]:
del train_graphs[0]
train_graphs

[Data(x=[260, 1159], edge_index=[2, 260], edge_attr=[260, 2], y=[260]),
 Data(x=[60, 1159], edge_index=[2, 60], edge_attr=[60, 2], y=[60]),
 Data(x=[246, 1159], edge_index=[2, 246], edge_attr=[246, 2], y=[246]),
 Data(x=[64, 1159], edge_index=[2, 64], edge_attr=[64, 2], y=[64]),
 Data(x=[286, 1159], edge_index=[2, 286], edge_attr=[286, 2], y=[286]),
 Data(x=[195, 1159], edge_index=[2, 195], edge_attr=[195, 2], y=[195]),
 Data(x=[94, 1159], edge_index=[2, 94], edge_attr=[94, 2], y=[94]),
 Data(x=[126, 1159], edge_index=[2, 126], edge_attr=[126, 2], y=[126]),
 Data(x=[148, 1159], edge_index=[2, 148], edge_attr=[148, 2], y=[148]),
 Data(x=[251, 1159], edge_index=[2, 251], edge_attr=[251, 2], y=[251]),
 Data(x=[160, 1159], edge_index=[2, 160], edge_attr=[160, 2], y=[160]),
 Data(x=[151, 1159], edge_index=[2, 151], edge_attr=[151, 2], y=[151]),
 Data(x=[166, 1159], edge_index=[2, 166], edge_attr=[166, 2], y=[166]),
 Data(x=[191, 1159], edge_index=[2, 191], edge_attr=[191, 2], y=[191]),
 Dat

In [39]:
for idx, val_graph in enumerate(val_graphs):
    if val_graph.x.shape[0] != val_graph.y.shape[0]:
        print(f"Abnormal at index: {idx}")
        print(f"Train graph's input shape: {val_graph['x'].shape[0]} and labels shape: {val_graph['y'].shape[0]}")

### Model Training

In [40]:
# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_graphs, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(val_graphs, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_graphs_data, batch_size=batch_size, shuffle=False)

### Customized loss function

In [41]:
class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, pos_weight):
        super().__init__()
        self.pos_weight = pos_weight

    def forward(self, logits, labels):
        # Handle class imbalance with weighted loss
        weight = torch.tensor([1.0, self.pos_weight]).to(logits.device)  # [weight for class 0, weight for class 1]
        loss_fct = nn.CrossEntropyLoss(weight=weight)
        return loss_fct(logits, labels)

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha  # Weight for the positive class
        self.gamma = gamma  # Focusing parameter
        self.reduction = reduction

    def forward(self, logits, labels):
        # Compute the cross-entropy loss (without reduction)
        ce_loss = nn.functional.cross_entropy(logits, labels, reduction='none')

        # Compute the probability of the true class
        probs = torch.softmax(logits, dim=-1)
        true_probs = probs[torch.arange(probs.size(0), device=probs.device), labels]

        # Compute the focal loss term: (1 - p_t)^gamma
        focal_term = (1 - true_probs) ** self.gamma

        # Apply the alpha weighting
        alpha_weight = torch.where(labels == 1, self.alpha, 1.0 - self.alpha).to(logits.device)

        # Compute the focal loss
        loss = alpha_weight * focal_term * ce_loss

        # Apply reduction
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

class PositionAwareLoss(nn.Module):
    def __init__(self, pos_weight, position_weight, alpha=0.25, gamma=2.0):
        super().__init__()
        self.weighted_ce = WeightedCrossEntropyLoss(pos_weight)
        self.focal_loss = FocalLoss(alpha=alpha, gamma=gamma, reduction='sum')
        self.position_weight = position_weight

    def forward(self, logits, labels, batch=None):
        # Compute the base loss (weighted cross-entropy + focal loss)
        ce_loss = self.weighted_ce(logits, labels)
        focal_loss = self.focal_loss(logits, labels)
        base_loss = ce_loss + focal_loss

        # Position-aware component
        probs = torch.softmax(logits, dim=-1)[:, 1]  # Get binding probabilities [total_num_nodes]
        position_loss = torch.tensor(0.0).to(logits.device)

        # Since we're using a GNN, we need to account for the graph structure
        # batch.batch indicates which nodes belong to which graph
        if batch is not None and batch.num_graphs == 1:  # Single graph per batch
            num_nodes = batch.num_nodes
            # Assume nodes are ordered sequentially (based on residue indices)
            # Penalize offset predictions by checking neighboring nodes
            for i in range(1, num_nodes - 1):
                # Encourage predictions to match true binding site positions
                if labels[i] == 1 or labels[i-1] == 1 or labels[i+1] == 1:
                    position_loss += torch.abs(probs[i] - (labels[i] == 1).float())

        return base_loss + self.position_weight * position_loss

In [42]:
model = BindingSiteGCN(node_dim=1159, edge_dim=2, hidden_dim=512).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# class_weights = torch.tensor([0.2, 0.8]).to(device)
pos_weight = 5.0
position_weight = 0.75
focal_loss = FocalLoss(alpha=0.25, gamma=2.0)
# criterion = WeightedCrossEntropyLoss(pos_weight=pos_weight)
criterion = PositionAwareLoss(pos_weight=pos_weight, position_weight=position_weight, alpha=0.25, gamma=2.0)

In [43]:
def evaluate(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for batch in data_loader:
            batch = batch.to(device)
            out = model(batch)

            # Debug: Print shapes
            # print(f"Batch num_graphs: {batch.num_graphs}")
            # print(f"Batch num_nodes: {batch.num_nodes}")
            # print(f"Out shape: {out.shape}")
            # print(f"Labels shape: {batch.y.shape}")

            probs = torch.softmax(out, dim=1)[:, 1]
            preds = torch.argmax(out, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch.y.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    auc = roc_auc_score(all_labels, all_probs)
    mcc = matthews_corrcoef(all_labels, all_preds)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "auc": auc,
        "mcc": mcc
    }

In [44]:
gc.collect()  # Force garbage collection to potentially free up memory
# torch.cuda.empty_cache()  # Empty the CUDA cache
# model = model.half()  # Convert model parameters to half-precision
# torch.cuda.synchronize()

50655

In [45]:
num_epochs = 300
best_val_f1 = 0
best_model_state = None

model.train()
for epoch in range(num_epochs):
    total_train_loss = 0
    model.train()
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        # print(f"Batch shape: {batch}")
        out = model(batch)
        # print(f"Out shape: {out.shape}")
        # print(f"Labels shape: {batch.y.shape}")
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    val_metrics = evaluate(model, eval_loader, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Training Loss: {avg_train_loss:.4f}")
    print(f"Validation Precision: {val_metrics['precision']:.4f}")
    print(f"Validation Recall: {val_metrics['recall']:.4f}")
    print(f"Validation F1-Score: {val_metrics['f1']:.4f}")
    print(f"Validation AUC-ROC: {val_metrics['auc']:.4f}")

    if val_metrics['f1'] > best_val_f1:
        best_val_f1 = val_metrics['f1']
        best_model_state = model.state_dict()
        print("Best validation F1-score improved! Saving model state.")
    # else:
    #     patience_counter += 1
    #     print(f"  Patience counter: {patience_counter}/{patience}")

    # if patience_counter >= patience:
    #     print("Early stopping triggered!")
    #     break

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/300
Training Loss: 114.2895
Validation Precision: 0.0000
Validation Recall: 0.0000
Validation F1-Score: 0.0000
Validation AUC-ROC: 0.5092


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 2/300
Training Loss: 79.1945
Validation Precision: 0.0000
Validation Recall: 0.0000
Validation F1-Score: 0.0000
Validation AUC-ROC: 0.5214


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

Epoch 7/300
Training Loss: 65.0730
Validation Precision: 0.0000
Validation Recall: 0.0000
Validation F1-Score: 0.0000
Validation AUC-ROC: 0.7891
Epoch 8/300
Training Loss: 63.5181
Validation Precision: 1.0000
Validation Recall: 0.0014
Validation F1-Score: 0.0028
Validation AUC-ROC: 0.7878
Best validation F1-score improved! Saving model state.


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 9/300
Training Loss: 64.2955
Validation Precision: 0.0000
Validation Recall: 0.0000
Validation F1-Score: 0.0000
Validation AUC-ROC: 0.7863


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 10/300
Training Loss: 63.5588
Validation Precision: 0.0000
Validation Recall: 0.0000
Validation F1-Score: 0.0000
Validation AUC-ROC: 0.7918
Epoch 11/300
Training Loss: 62.0601
Validation Precision: 1.0000
Validation Recall: 0.0014
Validation F1-Score: 0.0028
Validation AUC-ROC: 0.7952
Epoch 12/300
Training Loss: 62.5188
Validation Precision: 1.0000
Validation Recall: 0.0055
Validation F1-Score: 0.0110
Validation AUC-ROC: 0.8052
Best validation F1-score improved! Saving model state.
Epoch 13/300
Training Loss: 61.8959
Validation Precision: 1.0000
Validation Recall: 0.0083
Validation F1-Score: 0.0165
Validation AUC-ROC: 0.8062
Best validation F1-score improved! Saving model state.
Epoch 14/300
Training Loss: 61.0607
Validation Precision: 0.8000
Validation Recall: 0.0166
Validation F1-Score: 0.0325
Validation AUC-ROC: 0.8074
Best validation F1-score improved! Saving model state.
Epoch 15/300
Training Loss: 61.1095
Validation Precision: 1.0000
Validation Recall: 0.0083
Validation F1-

In [46]:
if best_model_state is not None:
    model.load_state_dict(best_model_state)
model.eval()  # Set the model to evaluation mode
test_metrics = evaluate(model, test_loader, device)

In [47]:
print(test_metrics)

{'precision': 0.4202434336963485, 'recall': 0.11177372635883455, 'f1': 0.1765814266487214, 'auc': 0.7184838482510525, 'mcc': 0.17917861983959726}


### Save and load model weights

In [48]:
# torch.save(best_model_state, '/content/drive/MyDrive/Protein-binding/trained_models/baseline_GCN_11Apr_model.pth')

In [49]:
# model_instance = BindingSiteGCN(node_dim=1287, edge_dim=2, hidden_dim=512).to(device)
# model_instance.load_state_dict(torch.load('/content/drive/MyDrive/Protein-binding/trained_models/baseline_GCN_10Apr_model.pth'))
# model_instance.eval()

# test_metrics = evaluate(model_instance, test_loader, device)
# print(test_metrics)

### Error analysis

In [50]:
all_preds, all_labels, all_probs = [], [], []

with torch.no_grad():
    for test_batch in test_loader:
        test_batch = test_batch.to(device)
        out = model(test_batch)

        # Debug: Print shapes
        # print(f"Batch num_graphs: {batch.num_graphs}")
        # print(f"Batch num_nodes: {batch.num_nodes}")
        # print(f"Out shape: {out.shape}")
        # print(f"Labels shape: {batch.y.shape}")

        probs = torch.softmax(out, dim=1)[:, 1]
        preds = torch.argmax(out, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch.y.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

In [51]:
all_test_prot_sequences = test_df['sequence'].tolist()
all_sequences = []
for prot_seq in all_test_prot_sequences:
    all_sequences.extend(list(prot_seq))

In [52]:
from collections import Counter
from copy import deepcopy

test_aa_counter = Counter(all_sequences)
false_negatives_dict = {}

for prob, label, pred, acid in zip(all_probs, all_labels, all_preds, all_sequences):
    if label == 1 and pred == 0:
        print(f"Acid: {acid} and its probability: {prob}")

        if acid not in false_negatives_dict:
            false_negatives_dict[acid] = 1
        else:
            false_negatives_dict[acid] += 1

Acid: L and its probability: 0.31487756967544556
Acid: L and its probability: 0.3357260227203369
Acid: D and its probability: 0.2273397445678711
Acid: G and its probability: 0.017061086371541023
Acid: S and its probability: 0.014831718988716602
Acid: G and its probability: 0.2167365998029709
Acid: L and its probability: 0.15380214154720306
Acid: R and its probability: 0.13569094240665436
Acid: E and its probability: 2.3476108253817074e-05
Acid: Y and its probability: 0.05887001007795334
Acid: R and its probability: 0.006887076422572136
Acid: Q and its probability: 0.005311434622853994
Acid: W and its probability: 0.037998054176568985
Acid: A and its probability: 0.00020161223073955625
Acid: T and its probability: 0.22173850238323212
Acid: A and its probability: 0.29981470108032227
Acid: V and its probability: 0.3624482750892639
Acid: R and its probability: 0.17428945004940033
Acid: N and its probability: 0.0012384640285745263
Acid: A and its probability: 0.006785335019230843
Acid: R an

In [53]:
false_negatives_percentage_dict = deepcopy(false_negatives_dict)

for acid in false_negatives_percentage_dict:
    false_negatives_percentage_dict[acid] /= test_aa_counter[acid]

In [54]:
# print(false_negatives_percentage_dict)
sorted_false_negatives_percentage_dict = dict(sorted(false_negatives_percentage_dict.items(), key=lambda item: item[1], reverse=True))
print(sorted_false_negatives_percentage_dict)

{'W': 0.002068252326783868, 'C': 0.002002002002002002, 'R': 0.0017079419299743809, 'I': 0.0013679890560875513, 'L': 0.0012239902080783353, 'V': 0.001218026796589525, 'N': 0.0010997067448680353, 'S': 0.0010579211848717272, 'G': 0.0009633911368015414, 'E': 0.0009229349330872173, 'Q': 0.0008326394671107411, 'P': 0.0006879944960440316, 'T': 0.0006244146113019045, 'A': 0.0006129955046996322, 'K': 0.0005337603416066186, 'Y': 0.0004723665564478035, 'F': 0.00039032006245121, 'D': 0.00029036004645760743}


In [55]:
sorted_false_negatives_percentage_dict

{'W': 0.002068252326783868,
 'C': 0.002002002002002002,
 'R': 0.0017079419299743809,
 'I': 0.0013679890560875513,
 'L': 0.0012239902080783353,
 'V': 0.001218026796589525,
 'N': 0.0010997067448680353,
 'S': 0.0010579211848717272,
 'G': 0.0009633911368015414,
 'E': 0.0009229349330872173,
 'Q': 0.0008326394671107411,
 'P': 0.0006879944960440316,
 'T': 0.0006244146113019045,
 'A': 0.0006129955046996322,
 'K': 0.0005337603416066186,
 'Y': 0.0004723665564478035,
 'F': 0.00039032006245121,
 'D': 0.00029036004645760743}